<a href="https://colab.research.google.com/github/soumyadip1995/BabyGPT/blob/main/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BabyGPT trainer.

In [50]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
from dataclasses import dataclass
from typing import Optional

# hyperparameters

@dataclass
class GPTConfig:
    # these are default GPT-2 hyperparameters
    block_size: int = 1024
    vocab_size: int = 50304
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    bias :bool = False


### other hyperparametres
batch_size = 64 
max_iters = 11000
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_interval = 500
eval_iters = 200
dropout = 0.2


torch.manual_seed(1337)
words = open(r"/content/ALL_eminem.txt", 'r', encoding='utf-8').read()

chars = sorted(list(set(words)))
vocab_size = len(chars)


string2integer = {ch: i for i, ch in enumerate(chars)}
integer2string = {i:ch for ch,i in string2integer.items()}
encode = lambda s: [string2integer[c] for c in s]
decode = lambda l: ''.join([integer2string[i] for i in l])
data = torch.tensor(encode(words), dtype = torch.long)


## train and split the data
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - config.block_size, (batch_size,))
    x = torch.stack([data[i:i+ config.block_size] for i in ix])
    y = torch.stack([data[i+1:i+ config.block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


### from pytorch GPT tutorial
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


torch.manual_seed(1337)
class Attention(nn.Module):
  def __init__(self, config):
    super(Attention, self).__init__()

    assert config.n_embd % config.n_head == 0

    self.atten = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
    self.projection = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
    self.n_head = config.n_head
    self.n_embd = config.n_embd
    self.register_buffer('tril', torch.tril(torch.ones(config.block_size, config.block_size)))

  def forward(self, x):
    B,T,C = x.size()
    q, k ,v  = self.atten(x).split(self.n_embd, dim=2)
    q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)


    # manual implementation of attention
    # from karpathy
    att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
    att = att.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    att = F.softmax(att, dim=-1)
    y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
    y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

    # output projection
    y = self.projection(y)
    return y

dropout = 0.2
class FeedForward(nn.Module):
  def __init__(self,config):
    super(FeedForward, self).__init__()
    self.net = nn.Sequential(nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias),
    nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias),
    nn.GELU(),
    nn.Dropout(dropout))

  def forward(self, x):
    return self.net(x)

### A simple Transformer Block    
class Transformer(nn.Module):
  def __init__(self,config):
    super(Transformer, self).__init__()
    self.attention = Attention(config)
    self.feed_forward = FeedForward(config)
    self.layer_norm_1 = nn.LayerNorm(config.n_embd)
    self.layer_norm_2 = nn.LayerNorm(config.n_embd)

  def forward(self, x):
    
    x = x + self.attention(self.layer_norm_1(x))
    x = x + self.feed_forward(self.layer_norm_2(x))
    return x


class BabyGPTmodel(nn.Module):

    def __init__(self, config):
        super(BabyGPTmodel, self).__init__()

        assert config.vocab_size is not None
        assert config.block_size is not None

        self.config = config
        self.token = nn.Embedding(config.vocab_size, config.n_embd)
        self.positional_embeddings = nn.Embedding(config.block_size, config.n_embd)
        self.blocks = nn.Sequential(*[Transformer(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd, eps = 1e-12) # final layer norm
        self.lnum_heads = nn.Linear(config.n_embd, config.vocab_size)

        ## init all weights
        ## from karpathy
        self.apply(self._init_weights)
        for pn, p in self.named_parameters():
          if pn.endswith('projection.weight'):
            torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %d" % (sum(p.nelement() for p in self.parameters()),))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

    def forward(self, idx, targets=None):
        device = idx.device
        B, T = idx.shape
        tok_emb = self.token(idx)
        position_ids = torch.arange(0, T, dtype = torch.long, device = device).unsqueeze(0)
        pos_emb =  self.positional_embeddings(position_ids)
        x = tok_emb + pos_emb
        for block in self.blocks:
          x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lnum_heads(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

        
    ## from karpathy's youtube videos.
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -config.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


config = GPTConfig(
    block_size = 4,
    vocab_size = len(chars),
    n_head = 4,
    n_layer = 4,
    n_embd = 16)

model = BabyGPTmodel(config)

m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


number of parameters: 16237
step 0: train loss 4.6938, val loss 4.6940
step 500: train loss 3.1403, val loss 3.1382
step 1000: train loss 2.7314, val loss 2.7349
step 1500: train loss 2.5625, val loss 2.5675
step 2000: train loss 2.4877, val loss 2.5002
step 2500: train loss 2.4302, val loss 2.4225
step 3000: train loss 2.4015, val loss 2.4092
step 3500: train loss 2.3725, val loss 2.3798
step 4000: train loss 2.3553, val loss 2.3565
step 4500: train loss 2.3320, val loss 2.3338
step 5000: train loss 2.3159, val loss 2.3222
step 5500: train loss 2.3110, val loss 2.3110
step 6000: train loss 2.2861, val loss 2.2992
step 6500: train loss 2.2905, val loss 2.2927
step 7000: train loss 2.2843, val loss 2.2974
step 7500: train loss 2.2793, val loss 2.2889
step 8000: train loss 2.2767, val loss 2.2738
step 8500: train loss 2.2616, val loss 2.2740
step 9000: train loss 2.2576, val loss 2.2935
step 9500: train loss 2.2596, val loss 2.2714
step 10000: train loss 2.2609, val loss 2.2769
step 1050

### llama trainer with MFU

In [51]:
## from lit-llama repo (partially).


import math
from dataclasses import dataclass
from typing import Optional
import time
import torch
import torch.nn as nn
from torch.nn import functional as F
from typing_extensions import Self

@dataclass
class LLaMAConfig:
    ## LLaMa parametres
    block_size: int = 2048
    vocab_size: int = 32000
    n_layer: int = 32
    n_head: int = 32
    n_embd: int = 4096


    @classmethod
    def from_name(cls, name: str) -> Self:
        return cls(**llama_configs[name])


llama_configs = {
    "7B": dict(n_layer=32, n_head = 32, n_embd=4096),
    "13B": dict(n_layer=40, n_head =40, n_embd=5120),
    "30B": dict(n_layer=60, n_head=52, n_embd=6656),
    "65B": dict(n_layer=80, n_head =64, n_embd=8192),
}


### other hyperparametres

batch_size = 64 
max_iters = 11000
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_interval = 500
eval_iters = 200
dropout = 0.2

words = open(r"/content/ALL_eminem.txt", 'r', encoding='utf-8').read()


chars = sorted(list(set(words)))
vocab_size = len(chars)


string2integer = {ch: i for i, ch in enumerate(chars)}
integer2string = {i:ch for ch,i in string2integer.items()}
encode = lambda s: [string2integer[c] for c in s]
decode = lambda l: ''.join([integer2string[i] for i in l])
data = torch.tensor(encode(words), dtype = torch.long)


## train and split the data
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - config.block_size, (batch_size,))
    x = torch.stack([data[i:i+ config.block_size] for i in ix])
    y = torch.stack([data[i+1:i+ config.block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


### from pytorch GPT tutorial
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Attention(nn.Module):
  def __init__(self, config : LLaMAConfig):
    super(Attention, self).__init__()

    self.config = config
    self.atten = nn.Linear(config.n_embd, 3 * config.n_embd)
    self.projection = nn.Linear(config.n_embd, config.n_embd)
    self.n_head = config.n_head
    self.n_embd = config.n_embd
    self.block_size = config.block_size
    self.rope_cache: Optional[torch.Tensor] = None
    self.register_buffer('tril', torch.tril(torch.ones(config.block_size, config.block_size)))

  def forward(self, x):
    B,T,C = x.size()
    q, k ,v  = self.atten(x).split(self.n_embd, dim=2)
    q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)


    if self.rope_cache is None:
      # cache for future forward calls
      self.rope_cache = build_rope_cache(
      seq_len=self.block_size,
      n_elem=self.n_embd // self.n_head, 
      dtype=x.dtype,
      device=x.device,
            )

      q = apply_rope(q, self.rope_cache)
      k = apply_rope(k, self.rope_cache)



    #manual implementation of attention
    #from karpathy
    # att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
    # att = att.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    # att = F.softmax(att, dim=-1)
    # y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
    # y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

    # efficient attention using Flash Attention CUDA kernels
    y = F.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=0.0, is_causal=True)

    y = y.transpose(1, 2).contiguous().view(B, T, C)  # re-assemble all head outputs side by side

    # output projection
    y = self.projection(y)

    return y

class FeedForward(nn.Module):
  def __init__(self, config: LLaMAConfig) -> None:
    super().__init__()
    hidden_dim = 4 * config.n_embd
    n_hidden = int(2 * hidden_dim / 3)


    self.c_fc1 = nn.Linear(config.n_embd, n_hidden, bias=False)
    self.c_fc2 = nn.Linear(config.n_embd, n_hidden, bias=False)
    self.c_proj = nn.Linear(n_hidden,  config.n_embd, bias=False)
    
  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = F.silu(self.c_fc1(x)) * self.c_fc2(x)
    x = self.c_proj(x)
    return x


### A simple Transformer Block    
class Transformer(nn.Module):
  def __init__(self, config : LLaMAConfig) -> None:
    super(Transformer, self).__init__()
    self.attention = Attention(config)
    self.feed_forward = FeedForward(config)
    self.layer_norm_1 = RMSNorm(config.n_embd)
    self.layer_norm_2 = RMSNorm(config.n_embd)

  def forward(self, x):
    
    x = x + self.attention(self.layer_norm_1(x))
    x = x + self.feed_forward(self.layer_norm_2(x))
    return x

class RMSNorm(nn.Module):
    """Root Mean Square Layer Normalization.

    Derived from https://github.com/bzhangGo/rmsnorm/blob/master/rmsnorm_torch.py. BSD 3-Clause License:
    https://github.com/bzhangGo/rmsnorm/blob/master/LICENSE.
    """

    def __init__(self, size: int, dim: int = -1, eps: float = 1e-5) -> None:
        super().__init__()
        self.scale = nn.Parameter(torch.ones(size))
        self.eps = eps
        self.dim = dim

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # NOTE: the original RMSNorm paper implementation is not equivalent
        # norm_x = x.norm(2, dim=self.dim, keepdim=True)
        # rms_x = norm_x * d_x ** (-1. / 2)
        # x_normed = x / (rms_x + self.eps)
        norm_x = torch.mean(x * x, dim=self.dim, keepdim=True)
        x_normed = x * torch.rsqrt(norm_x + self.eps)
        return self.scale * x_normed


def build_rope_cache(seq_len: int, n_elem: int, dtype: torch.dtype, device: torch.device, base: int = 10000) -> torch.Tensor:

    """Enhanced Transformer with Rotary Position Embedding.

    Derived from: https://github.com/labmlai/annotated_deep_learning_paper_implementations/blob/master/labml_nn/
    transformers/rope/__init__.py. MIT License:
    https://github.com/labmlai/annotated_deep_learning_paper_implementations/blob/master/license.
    """
    # $\Theta = {\theta_i = 10000^{\frac{2(i-1)}{d}}, i \in [1, 2, ..., \frac{d}{2}]}$
    theta = 1.0 / (base ** (torch.arange(0, n_elem, 2, dtype=dtype, device=device) / n_elem))

    # Create position indexes `[0, 1, ..., seq_len - 1]`
    seq_idx = torch.arange(seq_len, dtype=dtype, device=device)

    # Calculate the product of position index and $\theta_i$
    idx_theta = torch.outer(seq_idx, theta).float()

    cache = torch.stack([torch.cos(idx_theta), torch.sin(idx_theta)], dim=-1)

    # this is to mimic the behaviour of complex32, else we will get different results
    if dtype in (torch.float16, torch.bfloat16, torch.int8):
        cache = cache.half()
    return cache


def apply_rope(x: torch.Tensor, rope_cache: torch.Tensor) -> torch.Tensor:
    x = x.transpose(1, 2)

    # truncate to support variable sizes
    T = x.size(1)
    rope_cache = rope_cache[:T]

    # cast because the reference does
    xshaped = x.float().reshape(*x.shape[:-1], -1, 2)
    rope_cache = rope_cache.view(1, xshaped.size(1), 1, xshaped.size(3), 2)
    x_out2 = torch.stack(
        [xshaped[..., 0] * rope_cache[..., 0] - xshaped[..., 1] * rope_cache[..., 1],
         xshaped[..., 1] * rope_cache[..., 0] + xshaped[..., 0] * rope_cache[..., 1],
        ], -1)

    x_out2 = x_out2.flatten(3)
    return x_out2.transpose(1, 2).type_as(x)

class BabyGPTmodel(nn.Module):

    def __init__(self, config):
        super(BabyGPTmodel, self).__init__()

        assert config.vocab_size is not None
        assert config.block_size is not None

        self.config = config
        self.token = nn.Embedding(config.vocab_size, config.n_embd)
        self.positional_embeddings = nn.Embedding(config.block_size, config.n_embd)
        self.blocks = nn.Sequential(*[Transformer(config) for _ in range(config.n_layer)])
        self.ln_f = RMSNorm(config.n_embd, eps = 1e-12) # final layer norm
        self.lnum_heads = nn.Linear(config.n_embd, config.vocab_size)

        ## init all weights
        ## from karpathy
        self.apply(self._init_weights)
        for pn, p in self.named_parameters():
          if pn.endswith('attention.weight'):
            torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %d" % (sum(p.nelement() for p in self.parameters()),))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

    def num_params(self):
      n_params = sum(p.numel() for p in self.parameters())
      return n_params


    def model_flops(self, for_back, dt):
      # from https://arxiv.org/pdf/2204.02311.pdf section B
      cfg = self.config 
      N = self.num_params()
      
      H = cfg.n_head
      Q = cfg.n_embd // config.n_head
      T = cfg.block_size
      L = cfg.n_layer
      flops = 6*N + 12*L*H*Q*T
      flops_per_for_back = flops * T
      flops_per_iteration =  for_back * flops_per_for_back
      flops_received= flops_per_iteration * (1.0/dt) # per second
      theoretical_flops = 8e12  # tesla t4 has about 8.1 TFLOPS
      mfu = flops_received / theoretical_flops


      return mfu

    def forward(self, idx, targets=None):
        device = idx.device
        B, T = idx.shape
        tok_emb = self.token(idx)
        position_ids = torch.arange(0, T, dtype = torch.long, device = device).unsqueeze(0)
        pos_emb =  self.positional_embeddings(position_ids)
        x = tok_emb + pos_emb
        for block in self.blocks:
          x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lnum_heads(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

        
    ## from karpathy's youtube videos.
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -config.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


config = LLaMAConfig(
    block_size = 4,
    vocab_size = len(chars),
    n_head = 4,
    n_layer = 4,
    n_embd = 16)

model = BabyGPTmodel(config)

m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    t0 = time.time()
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    t1 = time.time()
    dt = t1 - t0

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
mfu = model.model_flops(batch_size * 1, dt)
print(mfu)
print(f" Model Flop Utilization: {mfu*100:.10f}%")

number of parameters: 16221
step 0: train loss 4.6939, val loss 4.6938
step 500: train loss 3.1954, val loss 3.2022
step 1000: train loss 2.7772, val loss 2.7765
step 1500: train loss 2.5829, val loss 2.5818
step 2000: train loss 2.4838, val loss 2.4917
step 2500: train loss 2.4284, val loss 2.4341
step 3000: train loss 2.3717, val loss 2.3847
step 3500: train loss 2.3438, val loss 2.3475
step 4000: train loss 2.3067, val loss 2.3082
step 4500: train loss 2.2932, val loss 2.2989
step 5000: train loss 2.2573, val loss 2.2875
step 5500: train loss 2.2691, val loss 2.2596
step 6000: train loss 2.2572, val loss 2.2504
step 6500: train loss 2.2560, val loss 2.2673
step 7000: train loss 2.2190, val loss 2.2486
step 7500: train loss 2.2344, val loss 2.2385
step 8000: train loss 2.2142, val loss 2.2441
step 8500: train loss 2.2079, val loss 2.2252
step 9000: train loss 2.2111, val loss 2.2165
step 9500: train loss 2.2205, val loss 2.2138
step 10000: train loss 2.2075, val loss 2.2270
step 1050

Both The models starts to converge towards the end, which can be fixed. (We need the loss value to reach local minima)


Due to the use of Cuda efficient attention, the llama tokens seem more readable. 



```
``` number of parameters: 16221 ```
step 0: train loss 4.6937, val loss 4.6937
step 500: train loss 3.1929, val loss 3.1976
step 1000: train loss 2.7869, val loss 2.7994
step 1500: train loss 2.5804, val loss 2.5644
step 2000: train loss 2.4740, val loss 2.4840
step 2500: train loss 2.4136, val loss 2.4224
step 3000: train loss 2.3908, val loss 2.3613
step 3500: train loss 2.3396, val loss 2.3496
step 4000: train loss 2.3144, val loss 2.3300
step 4500: train loss 2.2939, val loss 2.2975
step 5000: train loss 2.2758, val loss 2.2965
step 5500: train loss 2.2653, val loss 2.2642
step 6000: train loss 2.2356, val loss 2.2579
step 6500: train loss 2.2375, val loss 2.2483
step 7000: train loss 2.2300, val loss 2.2480
step 7500: train loss 2.2402, val loss 2.2513
step 8000: train loss 2.2317, val loss 2.2301
step 8500: train loss 2.2147, val loss 2.2079
step 9000: train loss 2.2162, val loss 2.2254
step 9500: train loss 2.1962, val loss 2.2249
step 10000: train loss 2.1895, val loss 2.2101
step 10500: train loss 2.1978, val loss 2.2040
step 10999: train loss 2.2015, val loss 2.2009

Oh a he spoments
Ohs and whet me.0
Jay, Dre hanf inche, buh, I'm herere a win'
So to saiidid
I I'mf neved, I donse efy fake'ind merile (I'd nobqu cad-shegeds..]
Get withank to I what baut the gudin' I'm no]
Now oun the cuckedid in
I gine we dey (Be be doup I cloe that in a treve I ack pith my!
I gine we dey (Be be doup I cloe that in a treve I ack pith my!
?
Thhide was ueves sueel The ray heB*Red, yeu the me won beall you cham?
This if to my ona um a lating ut Dof'
Just to to sarse mighior mey (namale me
Heats to ine Ack moy t'
She good whene core pruting yo
```





```
! git clone https://github.com/soumyadip1995/BabyGPT.git

```



### Model FLOP Utilization

MFU is the ratio of
the observed throughput (tokens-per-second), relative to the theoretical maximum throughput of a system operating
at peak FLOPs.

 L, H, Q, and T are the number of layers, the number of heads, the head dimension, and the
sequence length respectively

If a given group of accelerators have a total theoretical peak matmul throughput of P
FLOPs per second, the model FLOPs utilization is the ratio of the achieved throughput in tokens per second to the
theoretical peak throughput R =
P/
(6N + 12LHQT)
.

Tesla t4 has a theoritical peak matmul of 8.1 TFLOPS.
This can be calculated with a few lines.




```


    def model_flops(self, for_back, dt):
      # from https://arxiv.org/pdf/2204.02311.pdf section B
      cfg = self.config 
      N = self.num_params()
      
      H = cfg.n_head
      Q = cfg.n_embd // config.n_head
      T = cfg.block_size
      L = cfg.n_layer
      flops = 6*N + 12*L*H*Q*T
      flops_per_for_back = flops * T
      flops_per_iteration =  for_back * flops_per_for_back
      flops_received= flops_per_iteration * (1.0/dt) # per second
      theoretical_flops = 8e12  # tesla t4 has about 8.1 TFLOPS
      mfu = flops_received / theoretical_flops
```

